<a href="https://colab.research.google.com/github/rayshee254/PLP-FINAL-PROJECT/blob/main/Copy_of_Audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio openai-whisper transformers reportlab



In [ ]:
# Step 1: Import libraries
import gradio as gr
import whisper
from transformers import pipeline
import re
import tempfile
from datetime import datetime
import os
import socket
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas

# Step 2: Find an available port automatically
def find_available_port(start_port=7860, max_attempts=10):
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                return port
        except OSError:
            continue
    return start_port

# Step 3: Set cache directories
os.environ['HF_HOME'] = '/content/huggingface_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/huggingface_cache'

# Step 4: Load models
print("✅ Models already loaded from previous run!")
print("🔍 Reusing existing models to save time...")

try:
    transcriber = whisper.load_model("small")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6", device=0)
    print("🎯 Models successfully reused!")
except:
    print("⚠️ Could not reuse models, loading fresh...")
    transcriber = whisper.load_model("small")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6", device=0)

# Step 5: Processing functions
def chunk_text(text, max_words=400):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

def process_audio(audio_file, language="en"):
    if audio_file is None:
        return "", "", "❌ Please upload an audio file first"

    try:
        result = transcriber.transcribe(audio_file, language=language)
        transcript_raw = result["text"]
        transcript = "\n".join(re.split(r'(?<=[.!?]) +', transcript_raw))

        chunks = chunk_text(transcript_raw)
        summaries = []

        for chunk in chunks:
            input_length = len(chunk.split())
            max_len = min(142, max(30, int(input_length * 0.6)))
            min_len = min(30, max_len)

            summary_text = summarizer(
                chunk,
                max_length=max_len,
                min_length=min_len,
                do_sample=False
            )[0]['summary_text']
            summaries.append(summary_text.strip())

        # Format summary with bullet points
        summary = "\n".join([f"• {s}" for s in summaries])
        return transcript, summary, "✅ Processing complete!"

    except Exception as e:
        return "", "", f"❌ Error: {str(e)}"

def download_results(transcript, summary):
    if not transcript and not summary:
        return None, "❌ No content to download"

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"transcription_summary_{timestamp}.pdf"

    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        c = canvas.Canvas(tmp_file.name, pagesize=LETTER)
        width, height = LETTER
        text_object = c.beginText(40, height - 40)
        text_object.setFont("Helvetica", 10)

        text_object.textLine("TRANSCRIPTION & SUMMARY REPORT")
        text_object.textLine(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        text_object.textLine("=" * 50)
        text_object.textLine("FULL TRANSCRIPT:")
        for line in transcript.splitlines():
            text_object.textLine(line)
        text_object.textLine("=" * 50)
        text_object.textLine("AI SUMMARY:")
        for line in summary.splitlines():
            if line.strip():
                text_object.textLine(line)
        text_object.textLine("=" * 50)
        text_object.textLine("Generated with AI Audio Processor")

        c.drawText(text_object)
        c.save()

        return tmp_file.name, "✅ PDF ready for download"

# Step 6: Create interface
def create_interface():
    with gr.Blocks(title="🎤 AI Audio Processor", theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # 🎤 AI Audio Processor
        **Ready to use! Models already loaded ✅**
        """)

        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(
                    sources=["upload", "microphone"],
                    type="filepath",
                    label="Upload Audio File"
                )

                language = gr.Dropdown(
                    choices=["en", "es", "fr", "de", "it", "pt", "ru", "zh", "ja", "ko"],
                    value="en",
                    label="Audio Language"
                )

                process_btn = gr.Button("🚀 Process Audio", variant="primary")

            with gr.Column():
                transcript_output = gr.Textbox(
                    label="📝 Full Transcript",
                    lines=12,
                    show_copy_button=True
                )

                summary_output = gr.Textbox(
                    label="📋 AI Summary (Bullet Points)",
                    lines=10,
                    show_copy_button=True
                )

                status_output = gr.Textbox(
                    label="Status",
                    interactive=False
                )

                with gr.Row():
                    download_btn = gr.Button("📥 Download PDF", variant="secondary")
                    download_file = gr.File(
                        label="Download File",
                        visible=False
                    )

        process_btn.click(
            fn=process_audio,
            inputs=[audio_input, language],
            outputs=[transcript_output, summary_output, status_output]
        )

        download_btn.click(
            fn=download_results,
            inputs=[transcript_output, summary_output],
            outputs=[download_file, status_output]
        ).then(
            lambda: gr.File(visible=True),
            outputs=download_file
        )

    return demo

# Step 7: Main execution
if __name__ == "__main__":
    available_port = find_available_port(7860)
    print(f"🔍 Found available port: {available_port}")

    demo = create_interface()
    print("🚀 Launching AI Audio Processor...")

    try:
        public_url = demo.launch(share=True, server_port=available_port)
        print(f"🌐 Your public URL: {public_url}")

        from IPython.display import HTML
        display(HTML(f"""
        <div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
                    padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
            <h2>🎉 Success! Audio Processor is Live!</h2>
            <p><strong>🌐 Public URL:</strong> <a href="{public_url}" target="_blank" style="color: #ffdd00; text-decoration: none;">{public_url}</a></p>
            <p><strong>Status:</strong> Models already loaded - Ready to process!</p>
            <p><strong>Port:</strong> {available_port} (automatically selected)</p>
        </div>
        """))

    except Exception as e:
        print(f"❌ Launch failed: {e}")
        print("🔄 Trying with default settings...")
        public_url = demo.launch(share=True)
        print(f"🌐 Backup URL: {public_url}")